In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftshift, fftfreq
from util.data_io import read_sdriq_data
from util.plotting import compute_fft_plot, compute_fft_plot_from_sample_rate, compute_frequency_response
import pylab
import matplotlib.pyplot as plt

from util.demodulation import demodulate_signal

import scipy.io.wavfile as wavfile

In [ ]:
# Crop and save the data. It's too much!
fs = 2048000
center_frequency = 144.390e6
raw_file = "/Users/benjaminpattison/Desktop/aprs_test5.dat"
imported_signal = read_sdriq_data(raw_file,start_index_fraction=4.85/30,end_index_fraction=5.5/30)

In [ ]:

f,t,sxx = signal.spectrogram(imported_signal,fs=fs,nfft=16*1024,return_onesided=False)
sxx = fftshift(sxx, axes=0)
print(f"Spectrum is %i by %i"%(np.shape(sxx)[0],np.shape(sxx)[1]))
f_min,f_max=np.min(f),np.max(f)
t_min,t_max=np.min(t),np.max(t)
c = sxx>-200
if np.sum(c)>100:
    lwr_range = np.mean(sxx[c]-2.0*np.std(sxx[c]))
    upr_range = np.mean(sxx[c]+6.0*np.std(sxx[c]))
else:
    lwr_range = -100
    upr_range = -50




In [ ]:
# Get time
#t=data[:,:1]

# Get spectra
spec=np.transpose(sxx)

# Get extrema
fmin,fmax=np.min(f),np.max(f)
tmin,tmax=np.min(t),np.max(t)

# Set dynamic range
c=spec>-200.0
if np.sum(c)>100:
    vmin=np.mean(spec[c])-2.0*np.std(spec[c])
    vmax=np.mean(spec[c])+6.0*np.std(spec[c])
else:
    vmin=-100
    vmax=-50
    
cmap="viridis"
print("%s %.2f %.2f %.2f %.2f %.2f"%("test",np.mean(spec[c]),np.median(spec[c]),np.std(spec[c]),vmin,vmax))

# Make plot
plt.figure(figsize=(5,10))
plt.imshow(spec,origin='lower',aspect='auto',interpolation='None',extent=[fmin,fmax,tmin,tmax],vmin=vmin,vmax=vmax,cmap=cmap)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Time (seconds)")
plt.xlim((-10e3,10e3))
fig=plt.colorbar(aspect=50)
fig.set_label("Power (dB)")

In [ ]:
decimated_signal = signal.decimate(
    signal.decimate(imported_signal,8,ftype='fir')
    ,8,ftype='fir')
resampled_signal = signal.resample(
    decimated_signal,
    int(len(decimated_signal)*.75)
    )

In [ ]:
demodulated_packets = np.angle(
    np.conjugate(resampled_signal[:-1])*
    resampled_signal[1:]
    )

fm_demodulated_signal, demod_fs = demodulate_signal(imported_signal, sample_rate=fs, base_band_filter_cutoff=10e3, base_band_downsample_rate=100, audio_filter_cutoff=10e3, audio_downsample_rate=1)
wavfile.write("/Users/benjaminpattison/Desktop/test.wav", int(demod_fs), fm_demodulated_signal)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(demodulated_packets[:10000])
plt.title("Angles of delayed conj * signal")

plt.figure(figsize=(10,5))
plt.plot(fm_demodulated_signal[:10000])
plt.title("Demod using demod signal util")

In [ ]:
t20 = np.arange(-10,10,1)/20
mf1200 = np.exp(1j*2*np.pi*t20*1.0)
mf2200 = np.exp(1j*2*np.pi*t20*1.8)

cos_1200 = np.exp(2*np.pi*1200*t20) + np.exp(-2*np.pi*1200*t20)
sin_1200 = np.exp(2*np.pi*1200*t20 + np.pi/2) + np.exp(-2*np.pi*1200*t20 + np.pi/2)
cos_2200 = np.exp(2*np.pi*2200*t20) + np.exp(-2*np.pi*2200*t20)
sin_2200 = np.exp(2*np.pi*2200*t20 + np.pi/2) + np.exp(-2*np.pi*2200*t20 + np.pi/2)

packet_1200 = np.convolve(demodulated_packets,mf1200,mode='same')
packet_2200 = np.convolve(demodulated_packets,mf2200,mode='same')
packet_diff = abs(packet_2200) - abs(packet_1200)

packet_1200a = np.convolve(fm_demodulated_signal,mf1200,mode='same')
packet_2200a = np.convolve(fm_demodulated_signal,mf2200,mode='same')
packet_diffa = abs(packet_2200a) - abs(packet_1200a)

offset = 1
n3 = 512
n4 = 20
n1 = 517 - offset
n2 = n1 + n4*n3

plt.figure(figsize=(10,5))
plt.plot(packet_diff[n1:n2])

plt.figure(figsize=(10,5))
plt.plot(packet_diffa[n1:n2])

In [ ]:
print( np.size(packet_diff[n1:n2]) )
print(n3*n4)

boff = np.arange(-10,10)
eyed = np.reshape(packet_diff[n1:n2],(n4,n3),order='F')
eyeda = np.reshape(packet_diffa[n1:n2],(n4,n3),order='F')

plt.figure(figsize=(5,5))
plt.plot(boff,eyed)

plt.figure(figsize=(5,5))
plt.plot(boff,eyeda)

In [ ]:
bits = packet_diff[n1:n2] > 0
bitsa = packet_diffa[n1:n2] > 0

plt.figure(figsize=(20,5))
plt.plot(bits)
plt.figure(figsize=(20,5))
plt.plot(bitsa)